# Python Test

The following notebook applies selected steps from the structural modeling workflow. In doing so, it tests whether Python and Pygeostat (and its dependencies) are properly installed. Users are encouraged to execute this notebook with ```Kernel > Restart & Runall```, before comparing the in-line results with the provided results in PythonTest.html. 

The following notebook is comprised of 9 primary steps:

1. Initialize required packages, directories and parameters 
2. Load and inspect the surface data
3. Decluster and normal score transform
4. Variogram calculation and modeling
5. Simulation and back-transformation
6. Simulation checking 
7. Base surface calculation
8. Save project setting and clean the output files

## 1. Initialize required packages and parameters

In [1]:
import pygeostat as gs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
gs.__version__

'1.2.0'

### Parameters and default settings

In [2]:
# Grid definition
griddef = gs.GridDef(gridfl='input/griddef.txt')
gs.gsParams['data.griddef'] = griddef
print(repr(griddef))

# Number of realizations
nreal = 4
gs.gsParams['data.nreal'] = nreal
print('nreal = {}'.format(nreal))

# Set the fontsize and figure size
gs.set_style(custom={'font.size':12, 'figure.figsize':(5, 5)})

TypeError: GridDef.__init__() got an unexpected keyword argument 'gridfl'. Did you mean 'grid_file'?

### Directories and files

In [3]:
# Path to the CCG executables
exedir = 'input/'

# create the output directory
outdir = 'Output/'
gs.mkdir(outdir)

## 1. Load and inspect the surface data

### Load the data and note its attributes

In [4]:
dat = gs.DataFile('input/PRM_surface.dat')
print('columns = {}'.format(dat.columns.values))
print('x column = {}, y column = {}'.format(dat.x, dat.y))

columns = ['HoleID' 'X' 'Y' 'Top Elevation' 'Thickness' 'Base Elevation']
x column = X, y column = Y


### Summary statistics
The DataFile describe excludes special attributes.

In [5]:
dat.describe()

,Top Elevation,Thickness,Base Elevation
count,230.000000,230.000000,230.000000
mean,379.173739,50.096391,329.077348
std,2.607090,4.376842,4.571519
min,372.070000,37.730000,315.850000
25%,377.222500,47.502500,327.137500
50%,378.950000,49.390000,329.140000
75%,380.782500,51.900000,332.225000
max,386.330000,62.870000,340.870000


### Location map

In [6]:
fig, axes = plt.subplots(2, 2, figsize=(10, 10))
axes = axes.flatten()
for var, ax in zip(dat.variables, axes):
    gs.locmap(dat, var=var, cbar_label='m', title=var, ax=ax)
# Remove the unneeded fourth axis
axes[-1].remove()
fig.tight_layout()

AttributeError: module 'pygeostat' has no attribute 'locmap'

## 3. Decluster and Normal Score Transform

Declustering defines the spatially representative distribution of the Top Elevation and Thickness. These distributions are then used for the normal score transformation, generating conditioning data that is used for calculating normal score variograms and conditioning sequential Gaussian simulation.

Note that variograms are often calculated on data that are normal score transformed without declustering weights. Given the controversy of the subject, the more convenient method is selected for this course (since declustering with weights is absolutely required for conditioning).

### Decluster the data
All variables are homotopically sampled, so only one variable need be considered for declustering.


In [7]:
declus = gs.Program(program=exedir+'declus', getpar=True)

C:\Users\boaky\CCG\PythonTest\tmppmxdg8_h\declus.par has been copied to the clipboard


In [ ]:
# Cell size based on the data spacing study in the intro notebook
cellsize=90

parstr = """   Parameters for DECLUS
                  *********************

START OF PARAMETERS:
{datafl}                 -file with data
{xycol}   0   {varcol}   -  columns for X, Y, Z, and variable
-1.0e21     1.0e21       -  trimming limits
junk.out                 -file for summary output
{outfl}                  -file for output with data & weights
1.0   1.0                   -Y and Z cell anisotropy (Ysize=size*Yanis)
0                           -0=look for minimum declustered mean (1=max)
1  {cellsize}  {cellsize}               -number of cell sizes, min size, max size
5                           -number of origin offsets
"""
declus.run(parstr=parstr.format(datafl=dat.flname,
                                xycol=dat.gscol(dat.xyz),
                                varcol=dat.gscol('Top Elevation'),
                                outfl=outdir+'declus.out',
                                cellsize=cellsize))
gs.rmfile('junk.out')

### Load and inspect the declustering results

In the context of the upcoming modeling steps, Base Elevation is not considered a variable. Use of the notvariables kwarg on initialization excludes it from the variables attribute.

In [8]:
dat = gs.DataFile('Output/declus.out', notvariables='Base Elevation')
print('declustering weight = ', dat.wts)
print('variables = ', dat.variables)

FileNotFoundError: Output/declus.out does not exist!

In [9]:
gs.locmap(dat, var=dat.wts, title=dat.wts)

AttributeError: module 'pygeostat' has no attribute 'locmap'

### Normal score transform

In [10]:
unscore = gs.Program(program=exedir+'unscore', getpar=True)

C:\Users\boaky\CCG\PythonTest\tmpiel3lz3d\unscore.par has been copied to the clipboard


In [11]:
parstr = """   Parameters for NSCORE
                  *********************

START OF PARAMETERS:
{datafl}                  -file with data
{nvar} {varcols}          -  number of variables and columns
{wtcol}                   -  column for weight, 0 if none
0                         -  column for category, 0 if none
0                         -  number of records if known, 0 if unknown
-1.0e21   1.0e21          -  trimming limits
0                         -transform using a reference distribution, 1=yes
../histsmth/histsmth.out  -file with reference distribution.
1   2   0                 -columns for variable, weight, and category
1000                       -maximum number of quantiles, 0 for all
{outfl}                   -file for output
{trnfl}                   -file for output transformation table
"""

unscore.run(parstr=parstr.format(datafl = dat.flname,
                                     nvar=dat.nvar,
                                     varcols=dat.gscol(dat.variables),
                                     wtcol=dat.gscol(dat.wts),
                                     outfl = outdir+'nscore.out',
                                     trnfl = outdir+'nscore.trn'))

AttributeError: 'DataFile' object has no attribute 'wts'

### Load and inspect the normal score transformation result

Use the notvariables kwarg leads to isolation of the normal score variables as the dat_ns.variables attribute.

In [12]:
dat_ns = gs.DataFile(outdir+'nscore.out', 
                     notvariables=['Base Elevation']+dat.variables)
print('variables = ', dat_ns.variables)

FileNotFoundError: Output/nscore.out does not exist!

In [13]:
fig, axes = plt.subplots(2 , 2, figsize=(10, 10))
for var, ax in zip(dat.variables, axes[0]):
    gs.histplt(dat[var], wt=dat[dat.wts], ax=ax, stat_blk='minimal')
for var, ax in zip(dat_ns.variables, axes[1]):
    gs.histplt(dat_ns, var=var, wt=True, ax=ax, 
               xlim=(-3, 3), stat_blk='minimal')  
fig.tight_layout()

AttributeError: module 'pygeostat' has no attribute 'histplt'

## 4. Variogram Calculation and Modeling
Normal score variograms are calculated and modeled, before being input to sequential Gaussian simulation in the next section.

Please refer to the Boundary Modeling notebook for additional details about the Variogram object.

In [14]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

# Initialize a list that will store the variogram objects
vargs = []
for var, ax in zip(dat_ns.variables, axes):
    varg = gs.Variogram(dat_ns, var, ndim=2, ndir=1, omnihz=True, 
                        mute=True)
    
    # Variogram calculation
    varg.update_calcpars(nlags=10, lagdist=30.0, lagtol=20.0, azm=0, 
                         azmtol=90, bandhorz=10000, variostd=True)
    varg.varcalc()
    
    # Variogram modeling
    varg.update_modelpars(c0=0.01, it=[3, 3], nst=2)
    varg.fitmodel(maxiter=2500, sill=1.0)

    # Variogram plotting
    fig = varg.plot(model=True, titles=var, axes=ax, separate_plts=False)
    
    # Append in the list
    vargs.append(varg)
fig.tight_layout()

NameError: name 'dat_ns' is not defined

## 5. Simulation and Back-transformation
The Top Elevation and Thickness is simulated using sequential Gaussian simulation, before back-transforming to original units.

### Sequential Gaussian Simulation

In [15]:
simdir = outdir+'USGSIM/'
gs.mkdir(simdir)
usgsim = gs.Program(program=exedir+'usgsim', getpar=True)

C:\Users\boaky\CCG\PythonTest\tmpmgiaawv5\usgsim.par has been copied to the clipboard


In [16]:
parstr = """               Parameters for USGSIM
                              *********************

START OF MAIN:
1                             -number of realizations to generate, 0=kriging
2                             -number of variables being simulated
0                             -number of rock types to consider
{seed}                        -random number seed
{griddef}
{outfl}                       -file for simulation output
2                             -  output format: (0=reg, 1=coord, 2=binary)
impute.out                    -file for imputed values in case of heterotopic samples
0                             -debugging level: 0,1,2,3
sgsim.dbg                     -file for debugging output 

START OF SRCH:
25                            -number of data to use per variable
300 300   10                  -maximum search radii (hmax,hmin,vert)
0 0 0                         -angles for search ellipsoid
1                             -sort by distance (0) or covariance (1)
0 1 1                         -if sorting by covariance, indicate variogram rock type, head, tail to use

START OF VARG:
2                             -number of variograms
0  1  1                       -rock type, variable 1, variable 2
{varmodel1}
0  2  2                       -rock type, variable 1, variable 2
{varmodel2}

START OF DATA:
{datafl}                      -file with primary data
{xyzcols}  0  0  0             -  columns for X,Y,Z,wt,rock type
{varcols}                     -  columns for variables
1                             -  clip data to grid, 1=yes
1                             -  assign to the grid, 0=none, 1=nearest, 2=average
-8.0       1.0e21             -  trimming limits
"""
pars = dict(griddef=griddef, varmodel1=vargs[0].model, varmodel2=vargs[1].model,
            datafl=dat_ns.flname, xyzcols=dat_ns.gscol(dat_ns.xyz),
            varcols=dat_ns.gscol(dat_ns.variables)) 
callpars = []
seeds = gs.rseed_list(nreal, seed=23243)
for i, seed in enumerate(seeds):
    pars['seed'] = seed
    pars['outfl'] = outfl=simdir+'real{}.gsb'.format(i+1)
    callpars.append(dict(parstr=parstr.format(**pars)))
gs.runparallel(usgsim, callpars, nprocess=4, reportprogress=True)

NameError: name 'griddef' is not defined

### Inspect a Gaussian realization
This step is not strictly required, but is presented for demonstration.

In [17]:
# Read in the simulation to inspect
sim = gs.DataFile(simdir+'real1.gsb')

# Rename the simulation columns
sim.columns=dat_ns.variables

# Summary statistics
print('\nProperties of the realization:\n', sim.describe())

FileNotFoundError: Output/USGSIM/real1.gsb does not exist!

In [18]:
# The gs.subplots is useful when multiple panels are plotted that 
# should use the same colorbar
fig, axes = gs.subplots(1, 2, figsize=(10, 10), cbar_mode='single')
for var, ax in zip(dat_ns.variables, axes):
    gs.pixelplt(sim, var=var, vlim=(-3, 3), title=var+' Realization', ax=ax)

NameError: name 'dat_ns' is not defined

### Back-transformation
Note that ubacktr program only requires a prefix of the transformation table, before it infers the file name based on the number of variables and categories.

In [19]:
backdir = outdir+'UBACKTR/'
gs.mkdir(backdir)
ubacktr = gs.Program(program=exedir+'ubacktr', getpar=True)

C:\Users\boaky\CCG\PythonTest\tmpev8pun3m\ubacktr.par has been copied to the clipboard


In [20]:
parstr = """                  Parameters for UBACKTR
                  **********************
 
START OF PARAMETERS: 
{datafl}                    -file with simulated Gaussian variables (see Note6)
-7.0 1.0e21                 -  trimming limits
2                           -  number of variables
1 2                         -  columns for variables
0                           -number of rocktypes (NRT) (0 if none)
nofile.out                  -  file for simulated RTs (see Note1 and Note6)
5                           -  column for RT 
31 32 34 35 36 37           -  RT category codes (see Note2)
{nx} {ny} 1                 -nx, ny, nz (0=calculated)(see Note3)
1                           -number of realizations
{trnfl}                     -prefix for trans tables (see Note4 and Note7)
{outfl}                     -output file (see Note6)
"""
callpars = []
for i in range(nreal):
    mypars = dict(datafl=simdir+'real{}.gsb'.format(i+1),
                  nx=griddef.nx,
                  ny=griddef.ny,
                  trnfl=outdir+'nscore',
                  outfl=backdir+'real{}.gsb'.format(i+1))
    callpars.append(dict(parstr=parstr.format(**mypars)))
gs.runparallel(ubacktr, callpars)

# Remove the Gaussian realizations since they're no longer needed
gs.rmdir(simdir)

NameError: name 'nreal' is not defined

### Realization maps
Generate a figure for each variable, where a single color bar is used for multiple realizations.

In [21]:
for var in dat.variables:
    fig, axes = gs.subplots(2, 2, figsize=(10, 8), cbar_mode='single')
    for real, ax in enumerate(axes):
        sim = gs.DataFile(backdir+'real{}.gsb'.format(real+1))
        sim.columns = dat.variables
        gs.pixelplt(sim, var=var, title='Realization {}'.format(real+1),
                    pointdata=dat, cbar_label='m', ax=ax)

    # Label the figure
    fig.suptitle(var, **{'weight':'bold'})

FileNotFoundError: Output/UBACKTR/real1.gsb does not exist!

## 6. Simulation Checking

Check that the realizations reproduce the histogram and variogram of the data.

### Histogram reproduction
Use of a '*' wildcard leads the histpltsim function to assume 1,...,nreal files are present.

In [22]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
for i, (var, ax) in enumerate(zip(dat.variables, axes)):
    gs.histpltsim(backdir+'real*.gsb', dat, refvar=var, 
                  refwt=True, refclr='r',
                  simvar=i+1, sim_fltype='gsb', title=var, ax=ax)

AttributeError: module 'pygeostat' has no attribute 'histpltsim'

### Variogram reproduction

The variogram of the data in original units must be calculated first, since the normal score variogram was previously calculated.

In [23]:
# Calculate the experimental data variograms
vargs = []
for var, ax in zip(dat_ns.variables, axes):
    varg = gs.Variogram(dat_ns, var, ndim=2, ndir=1, omnihz=True, 
                        mute=True)
    varg.update_calcpars(nlags=10, lagdist=30.0, lagtol=20.0, azm=0, 
                         azmtol=90, bandhorz=10000, variostd=True)
    varg.varcalc()    
    vargs.append(varg)

NameError: name 'dat_ns' is not defined

In [24]:
# Calculate the variograms of the realizations
for i, varg in enumerate(vargs):
    varg.update_simpars(datafl=backdir+'real*.gsb', nvar=1, 
                        varcols=i+1)
    varg.varsim(maxlags=50, vargrng=500, nprocess=4)

In [25]:
gs.gsParams['config.ignore_mpl_warnings'] = True

AttributeError: module 'pygeostat' has no attribute 'gsParams'

In [26]:
# Plot the variograms
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
for var, varg, ax in zip(dat.variables, vargs, axes):
    fig = varg.plot(sim=True, titles=var, figsize=(8, 3), axes=ax)
fig.tight_layout()

## 7. Construct the Base Surface Realizations
Subtract the thickness from the top elevation, providing the base elevation realizations. Output all values within a single directory.

In [27]:
surfdir='Surfaces/'
gs.mkdir(surfdir)
for real in range(nreal):
    sim = gs.DataFile(backdir+'real{}.gsb'.format(real+1))
    sim.columns=['Top Elevation', 'Thickness']
    sim['Base Elevation'] = sim['Top Elevation'] - sim['Thickness']
    sim.writefile(surfdir+'real{}.gsb'.format(real+1))

NameError: name 'nreal' is not defined

In [28]:
import matplotlib as mpl

In [29]:
mpl.__version__

'3.10.0'

In [30]:
var = 'Base Elevation'
fig, axes = gs.subplots(2, 2, figsize=(10, 8), cbar_mode='single')
for real, ax in enumerate(axes):
    sim = gs.DataFile(surfdir+'real{}.gsb'.format(real+1))
    gs.pixelplt(sim, var=var, title='Realization {}'.format(real+1),
                pointdata=dat, cbar_label='m', ax=ax)
fig.suptitle(var, **{'weight':'bold'})

FileNotFoundError: Surfaces/real1.gsb does not exist!

### VTK visualization of the top and base surfaces

Output coordinates will be in single precision 'float32' following the gsParams setting below (may be problematic with large utms). This is used here to conserve output file size.

When the DataFile is initialized, it is registered as a structured grid (dftype='sgrid') with specified z coordinates. 

Use of a vtk extension in writefile leads to VTK output, where x and y coordinates are assumed to follow the regular grid, whereas z follows irregular coordinates. Note that at least one coordinate must be irregular for sgrid to register as valid.

In [31]:
gs.gsParams['data.write_vtk.cdtype'] = 'float32'
for var in ['Base Elevation', 'Top Elevation']:
    sim = gs.DataFile(surfdir+'real1.gsb', z=var, dftype='sgrid')
    sim.writefile('{}_real1.vtk'.format(var), variables=[var])

AttributeError: module 'pygeostat' has no attribute 'gsParams'

## 8. Cleaning Directories and Files

In [32]:
gs.rmdir([outdir, surfdir])
gs.rmfile('temp')